In [1]:
import pandas as pd
import ta
import matplotlib.pyplot as plt
import numpy as np

In [2]:
gaz = pd.read_csv('Data/Dutch TTF Natural Gas Futures - Données Historiques (1).csv')
gaz['Date']=pd.to_datetime(gaz['Date'])
lists = ['Dernier', 'Ouv.', ' Plus Haut', 'Plus Bas', 'Vol.','Variation %']
for i in lists: 
    gaz[i] = gaz[i].str.replace(',', '.')
gaz.columns

/var/folders/9z/p9wc5ctx14z50221phyqkyz80000gn/T/ipykernel_55714/1168937049.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  gaz['Date']=pd.to_datetime(gaz['Date'])


Index(['Date', 'Dernier', 'Ouv.', ' Plus Haut', 'Plus Bas', 'Vol.',
       'Variation %'],
      dtype='object')

In [3]:
gaz['Dernier'] = pd.to_numeric(gaz['Dernier'], errors='coerce')

gaz['rsi'] = ta.momentum.rsi(gaz['Dernier'], window=14)
gaz['macd'] = ta.trend.macd_diff(gaz['Dernier'])
gaz['ema20'] = ta.trend.ema_indicator(gaz['Dernier'], window=20)
gaz['ema50'] = ta.trend.ema_indicator(gaz['Dernier'], window=50)

gaz

,Date,Dernier,Ouv.,Plus Haut,Plus Bas,Vol.,Variation %,rsi,macd,ema20,ema50
0,2024-03-15,27.050,26.255,27.085,26.000,0.08K,3.84%,NaN,NaN,NaN,NaN
1,2024-03-14,26.050,24.675,26.600,24.675,0.51K,4.60%,NaN,NaN,NaN,NaN
2,2024-03-13,24.905,25.100,25.425,24.555,0.24K,0.52%,NaN,NaN,NaN,NaN
3,2024-03-12,24.775,25.010,25.010,24.355,0.30K,-0.62%,NaN,NaN,NaN,NaN
4,2024-03-11,24.930,25.970,26.465,24.760,0.45K,-5.53%,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1612,2017-10-27,18.150,18.150,18.150,18.150,NaN,0.44%,34.299402,-0.128800,19.074238,19.420546
1613,2017-10-26,18.070,18.070,18.070,18.070,NaN,-0.22%,33.531057,-0.117551,18.978597,19.367584
1614,2017-10-25,18.110,18.110,18.110,18.110,NaN,0.84%,34.323259,-0.098296,18.895873,19.318267
1615,2017-10-24,17.960,17.960,17.960,17.960,NaN,-0.72%,32.747079,-0.086934,18.806742,19.265001


In [4]:
def convert_k_m_to_numeric(value):
    """
    Convert values with 'K' or 'M' suffix to float numbers.
    Args:
    - value: The string or numeric value to convert.
    
    Returns:
    - The converted value as float if 'K' or 'M' was found; otherwise, the original value.
    """
    if isinstance(value, str):  # Only process strings
        if value.endswith('K'):
            return float(value[:-1]) * 1e3
        elif value.endswith('M'):
            return float(value[:-1]) * 1e6
    return value

In [5]:
gaz['Vol.'] = gaz['Vol.'].apply(convert_k_m_to_numeric)
for column in gaz.columns:
    gaz[column] = pd.to_numeric(gaz[column], errors='coerce')
assert gaz.applymap(np.isreal).all().all(), "Non-numeric data found in the dataset."

/var/folders/9z/p9wc5ctx14z50221phyqkyz80000gn/T/ipykernel_55714/3463237170.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  assert gaz.applymap(np.isreal).all().all(), "Non-numeric data found in the dataset."


In [6]:
class FeatureEngineer:
    def __init__(self, data):
        self.data = data

    def calculate_technical_indicators(self):
        """
        Calculate specified technical indicators for the dataset.
        """
        self.data['rsi'] = ta.momentum.rsi(self.data['Dernier'], window=14)
        self.data['macd'] = ta.trend.macd_diff(self.data['Dernier'])
        self.data['ema20'] = ta.trend.ema_indicator(self.data['Dernier'], window=20)
        self.data['ema50'] = ta.trend.ema_indicator(self.data['Dernier'], window=50)

    def normalize_features(self):
        """
        Normalize features to have a similar scale.
        """
        from sklearn.preprocessing import MinMaxScaler
        scaler = MinMaxScaler()
        features = ['rsi', 'macd', 'ema20', 'ema50']  # Specify the features to normalize
        self.data[features] = scaler.fit_transform(self.data[features])

    def integrate_economic_indicators(self, economic_data):
        """
        Integrate economic indicators with market data.
        This function assumes economic_data is a DataFrame where columns are indicators and rows align with self.data's timeline.
        """
        self.data = pd.concat([self.data, economic_data], axis=1)

    def construct_feature_vector(self):
        """
        Combine all features into a single vector for each timestep.
        Assuming all necessary features are already columns in self.data,
        this function will return a numpy array representation of the DataFrame.
        """
        return self.data.values

feature_engineer = FeatureEngineer(gaz)

feature_engineer.calculate_technical_indicators()  # Calculate RSI, MACD, EMA20, EMA50

feature_engineer.normalize_features()  # Normalize the features
features_vector = feature_engineer.construct_feature_vector()  # Get the feature vector

In [7]:
# make a copy of gaz and fill NaN values with 0
gaz_copy = gaz.copy()
gaz_copy.fillna(0, inplace=True)

gaz_copy.head()

,Date,Dernier,Ouv.,Plus Haut,Plus Bas,Vol.,Variation %,rsi,macd,ema20,ema50
0,1710460800000000000,27.050,26.255,27.085,26.000,80.0,0.0,0.0,0.0,0.0,0.0
1,1710374400000000000,26.050,24.675,26.600,24.675,510.0,0.0,0.0,0.0,0.0,0.0
2,1710288000000000000,24.905,25.100,25.425,24.555,240.0,0.0,0.0,0.0,0.0,0.0
3,1710201600000000000,24.775,25.010,25.010,24.355,300.0,0.0,0.0,0.0,0.0,0.0
4,1710115200000000000,24.930,25.970,26.465,24.760,450.0,0.0,0.0,0.0,0.0,0.0


In [8]:
import numpy as np
import pandas as pd

class TradingEnvironment:
    def __init__(self, data, initial_balance=10000, transaction_cost=0.001):
        self.data = data
        self.state_space = data.shape[1]  # This should match the number of features used to represent a state
        self.action_space = 3  # For example: buy, sell,
        self.initial_balance = initial_balance
        self.transaction_cost = transaction_cost
        self.reset()

    def reset(self):
        self.current_step = 0
        self.balance = self.initial_balance
        self.portfolio_value = self.initial_balance
        self.done = False
        self.position = 0
        self.history = []  # To store trade history
        return self._next_observation()

    def _next_observation(self):
        return self.data.iloc[self.current_step]

    def step(self, action):        
        # Ensure action is within a valid range
        action = np.clip(action, -1, 1)

        # Calculate the number of shares bought/sold based on the action
        delta_position = action * self.balance  # This assumes all-in on each action

        # Get the current price from the dataset to calculate changes in portfolio value
        current_price = self.data.iloc[self.current_step]['Dernier']  # Assuming 'close' is a column in your dataset
        next_step = min(self.current_step + 1, len(self.data) - 1)  # Ensure we don't go past the end of the dataset
        next_price = self.data.iloc[next_step]['Dernier']

        # Update position and balance
        change_in_value = delta_position * (next_price - current_price) / current_price
        self.balance += change_in_value - (abs(delta_position) * self.transaction_cost)
        self.portfolio_value = self.balance  # This could be more complex if managing multiple positions
        self.position += delta_position

        self.current_step = next_step

        # Check if we're at the end
        if self.current_step >= len(self.data) - 1:
            self.done = True

        # Here, the reward could be the change in portfolio value, or some other metric
        reward = change_in_value - (abs(delta_position) * self.transaction_cost)
        
        # Record this step
        self.history.append((self.current_step, self.position, self.portfolio_value, reward))

        return self._next_observation(), reward, self.done, {}

    def render(self):
        print("Step:", self.current_step)
        print("Balance:", self.balance)
        print("Position:", self.position)
        print("Portfolio Value:", self.portfolio_value)

    def run_backtest(self, policy):
        """
        Run a full backtest of the trading environment using the provided policy.
        The policy function should take a state as input and return an action.
        """
        self.reset()
        while not self.done:
            current_state = self._next_observation()
            action = policy(current_state)
            self.step(action)
        return self.history

# Example usage:

# Define a simple policy function (replace this with your PPO agent's policy for real testing)
def sample_policy(state):
    # This is a dummy policy that randomly decides to buy, hold, or sell
    return np.random.uniform(-1, 1)

# Assuming 'gaz_data' is your preprocessed DataFrame including 'Dernier' prices and any features
env = TradingEnvironment(gaz)
backtest_history = env.run_backtest(sample_policy)


In [9]:
# !pip install torchinfo

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
from itertools import chain
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
import numpy as np
from torchinfo import summary


2024-03-23 16:39:58.172451: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
class Actor(nn.Module):
    def __init__(self, state_size, action_size, action_bound):
        super(Actor, self).__init__()
        self.action_bound = action_bound
        self.network = nn.Sequential(
            nn.Linear(state_size, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, action_size),
            nn.Tanh()
        )
    
    def forward(self, state):
        return self.action_bound * self.network(state)


class Critic(nn.Module):
    def __init__(self, state_size):
        super(Critic, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(state_size, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1)  # Output is a single value representing V(s)
        )
    
    def forward(self, state):
        return self.network(state)


In [12]:
class PPOAgent:
    def __init__(self, state_size, action_size, action_bound, lr_actor=1e-4, lr_critic=1e-3, action_std=0.5 , update_epochs=10 , clip_param=0.2 , entropy_beta=0.01):
        self.actor = Actor(state_size, action_size, action_bound)
        self.critic = Critic(state_size)
        self.optimizer_actor = optim.Adam(self.actor.parameters(), lr=lr_actor)
        self.optimizer_critic = optim.Adam(self.critic.parameters(), lr=lr_critic)
        self.action_std = action_std  # Define the standard deviation for action distribution
        self.gamma = 0.99
        self.tau = 0.95
        self.update_epochs = update_epochs
        self.clip_param = clip_param
        self.entropy_beta = entropy_beta

    def preprocess_state(self, state):
        return torch.FloatTensor(state).unsqueeze(0)

    def predict_action(self, state, return_log_prob=False):
        state = self.preprocess_state(state)
        with torch.no_grad():
            action_mean = self.actor(state)
            dist = torch.distributions.Normal(action_mean, self.action_std)  # Assuming self.action_std is defined
            action = dist.sample()
            log_prob = dist.log_prob(action) if return_log_prob else None
        return action.cpu().numpy(), log_prob


    def compute_gae(self, next_value, rewards, masks, values):
        gae = 0
        returns = []
        for step in reversed(range(len(rewards))):
            delta = rewards[step] + self.gamma * next_value * masks[step] - values[step]
            gae = delta + self.gamma * self.tau * masks[step] * gae
            next_value = values[step]
            returns.insert(0, gae + values[step])
        return returns


    def update_policy(self, states, actions, rewards, next_states, dones, old_log_probs):
        # Normalize rewards
        rewards = (rewards - np.mean(rewards)) / (np.std(rewards) + 1e-10)
        
        states = torch.FloatTensor(states)
        actions = torch.FloatTensor(actions)
        rewards = torch.FloatTensor(rewards)
        next_states = torch.FloatTensor(next_states)
        dones = torch.FloatTensor(dones)
        old_log_probs = torch.stack(old_log_probs)
        

        for _ in range(self.update_epochs):
            log_probs, state_values, dist_entropy = self.evaluate_actions(states, actions)
            advantages, returns = self.calculate_advantages(rewards, states, next_states, dones, state_values)

            # Calculate actor loss
            # print("log_probs.shape:", log_probs.shape)
            old_log_probs = old_log_probs.squeeze()
            old_log_probs = old_log_probs.mean(dim=-1).squeeze()
            # print("old_log_probs.shape:", old_log_probs.shape)
            ratio = torch.exp(log_probs - old_log_probs.detach())

            # print("ratio.shape:", ratio.shape)
            # print("advantages.shape:", advantages.shape)

            surr1 = ratio * advantages
            surr2 = torch.clamp(ratio, 1.0 - self.clip_param, 1.0 + self.clip_param) * advantages
            actor_loss = -torch.min(surr1, surr2).mean() - self.entropy_beta * dist_entropy.mean()

            # Calculate critic loss
            critic_loss = (returns - state_values).pow(2).mean()

            # Perform backprop
            self.optimizer_actor.zero_grad()
            actor_loss.backward()
            torch.nn.utils.clip_grad_norm_(self.actor.parameters(), max_norm=1.0)
            self.optimizer_actor.step()

            self.optimizer_critic.zero_grad()
            critic_loss.backward()
            torch.nn.utils.clip_grad_norm_(self.critic.parameters(), max_norm=1.0)
            self.optimizer_critic.step()

    def calculate_advantages(self, rewards, states, next_states, dones, state_values):
        with torch.no_grad():
            aggregated_rewards = rewards.mean(dim=-1).squeeze()
            next_state_values = self.critic(next_states).squeeze()
            

            # print("Before deltas computation")

            state_values = state_values.squeeze()

            # print("aggregated_rewards.shape:", aggregated_rewards.shape)
            # print("next_state_values.shape:", next_state_values.shape)
            # print("dones.shape:", dones.shape)
            # print("state_values.shape:", state_values.shape)
            

            deltas = aggregated_rewards + self.gamma * next_state_values * (1 - dones) - state_values


            # print("After deltas computation")




            advantages = torch.zeros_like(aggregated_rewards)
            running_add = 0.0
            for t in reversed(range(len(aggregated_rewards))):
                # print("deltas[t].shape:", deltas[t].shape)
                # print("self.gamma:", self.gamma)
                # print("self.tau:", self.tau)
                # print("dones[t].shape:", dones[t].shape)
                # print("running_add:", running_add)
                # print("advantages[t].shape:", advantages[t].shape)

                running_add = deltas[t] + self.gamma * self.tau * (1 - dones[t]) * running_add
                
                # print("running_add.shape after computation:", running_add.shape)
                advantages[t] = running_add

            returns = advantages + state_values
            advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-10)
            return advantages, returns




    def evaluate_actions(self, states, actions):
        states_np = np.array(states)  # Convert list to numpy array
        states = torch.FloatTensor(states_np)  # Then convert to tensor

        actions_np = np.array(actions)  # Convert list to numpy array
        actions = torch.FloatTensor(actions_np).squeeze(1)  # Then convert to tensor
        # actions = torch.FloatTensor(actions).squeeze(1)

        action_means = self.actor(states)  # Expected shape: [1616, 3]
        state_values = self.critic(states).squeeze(-1)  # Ensure [1616]
        
        dist = torch.distributions.Normal(action_means, self.action_std)

        # Calculate log probabilities and ensure it's reduced to [1616]
        # print("actions.shape:", actions.shape)
        log_probs = dist.log_prob(actions).sum(dim=-1)  # Summing over action dimensions

        # Calculate entropy and ensure it's already correctly shaped [1616]
        entropy = dist.entropy().sum(dim=-1)

        # print("log_probs.shape:", log_probs.shape)  # Should be [1616]
        # print("state_values.shape:", state_values.shape)  # Should be [1616]
        # print("entropy.shape:", entropy.shape)  # Should be [1616]

        return log_probs, state_values, entropy


    

# Define the environment and PPO agent parameters
state_size = 100  # Assume 100 features for the state
action_size = 1  # Buy/sell quantity as a single action
action_bound = 1  # Action limit (e.g., normalized quantity between -1 and 1)

# Instantiate the PPOAgent
agent = PPOAgent(state_size=env.state_space, action_size=env.action_space, action_bound=1)

# Now you can access actor and critic through the agent object
actor_model = Actor(state_size=100, action_size=1, action_bound=1)
critic_model = Critic(state_size=100)

# summary(actor_model, input_size=(1, 100))
# summary(critic_model, input_size=(1, 101))

# critic_model.summary()
print(actor_model)
print(critic_model)

Actor(
  (network): Sequential(
    (0): Linear(in_features=100, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
    (5): Tanh()
  )
)
Critic(
  (network): Sequential(
    (0): Linear(in_features=100, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)


In [13]:
import wandb
from tqdm import tqdm

def train_agent(env, agent, episodes=1000):
    with torch.autograd.set_detect_anomaly(True):
        for episode in tqdm(range(episodes)):
            state = env.reset()
            done = False
            states, actions, rewards, next_states, dones, old_log_probs = [], [], [], [], [], []
            while not done:
                action, log_prob = agent.predict_action(state, return_log_prob=True)  # Updated call


                next_state, reward, done, _ = env.step(action)
                
                # Store experiences
                states.append(state)
                actions.append(action)
                rewards.append(reward)
                next_states.append(next_state)
                dones.append(done)
                old_log_probs.append(log_prob)
                
                state = next_state

            # After collecting experience, update the policy
            agent.update_policy(states, actions, rewards, next_states, dones, old_log_probs)

            # print total rewards for the episode
            total_rewards = sum(sum(rewards)[0])
            wandb.log({"total_rewards": total_rewards})

            # print(f"Episode {episode + 1}: Total Rewards: {total_rewards}")

        print("Training complete")


In [14]:
def run_simulation(env, agent):
    state = env.reset()
    done = False
    total_pnl = 0
    print("Date\t\tAction\tReward\tPortfolio Value")

    while not done:
        action = agent.predict_action(state)
        next_state, reward, done, _ = env.step(action)

        # Action interpretation for logging: -1 (Sell), 1 (Buy), 0 (Hold)
        action_str = "Buy" if action > 0 else "Sell" if action < 0 else "Hold"

        # Assuming 'date' is part of the environment's state
        date = state.index[env.current_step].strftime('%Y-%m-%d')
        print(f"{date}\t{action_str}\t{reward:.2f}\t{env.portfolio_value:.2f}")

        total_pnl += reward
        state = next_state


    print(f"Final PnL: {total_pnl:.2f}")


In [15]:
import wandb

wandb.init(project="RL Trading", config={
    "episodes": 1000,
    "learning_rate_actor": 1e-4,
    "learning_rate_critic": 1e-3,
    # Add other relevant configuration parameters here
})
config = wandb.config


env = TradingEnvironment(gaz_copy)
agent = PPOAgent(state_size=env.state_space, action_size=env.action_space, action_bound=1)

# Proceed with training and running the simulation
train_agent(env, agent)
run_simulation(env, agent)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pnikitits (pnikitits_1). Use `wandb login --relogin` to force relogin


  0%|          | 0/1000 [00:00<?, ?it/s]/var/folders/9z/p9wc5ctx14z50221phyqkyz80000gn/T/ipykernel_55714/2407987356.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return torch.FloatTensor(state).unsqueeze(0)
/var/folders/9z/p9wc5ctx14z50221phyqkyz80000gn/T/ipykernel_55714/2407987356.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  states = torch.FloatTensor(states)
/var/folders/9z/p9wc5ctx14z50221phyqkyz80000gn/T/ipykernel_55714/2407987356.py:43: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() be

Training complete
Date		Action	Reward	Portfolio Value


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.